In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread

import zipfile
import os

# import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from tensorflow. keras.applications import VGG16

from tensorflow.keras.models import Sequential 
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, MaxPooling2D, Dropout, Flatten, BatchNormalization, Conv2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

2024-06-02 21:18:14.497645: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-02 21:18:14.497755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-02 21:18:14.664015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Обробка даних

In [2]:
# Розархівування

direct = "/kaggle/working/"


with zipfile.ZipFile(r"/kaggle/input/dogs-vs-cats/train.zip", "r") as zip_ref:
  zip_ref.extractall(direct)

with zipfile.ZipFile(r"/kaggle/input/dogs-vs-cats/test1.zip", "r") as zip_ref:
  zip_ref.extractall(direct)


In [ ]:
# Видалення папок

import shutil


shutil.rmtree("/kaggle/working/train")
shutil.rmtree("/kaggle/working/test1")

In [3]:
img_train = os.listdir(f"{direct}/train")
print(img_train[:20])

img_test = os.listdir(f"{direct}/test1")
print(img_test[:10])

['cat.11626.jpg', 'dog.10329.jpg', 'dog.663.jpg', 'cat.3425.jpg', 'dog.4160.jpg', 'cat.2098.jpg', 'cat.2888.jpg', 'dog.3972.jpg', 'cat.11999.jpg', 'dog.6257.jpg', 'dog.8872.jpg', 'cat.156.jpg', 'cat.4931.jpg', 'cat.9109.jpg', 'cat.11211.jpg', 'dog.3046.jpg', 'dog.1380.jpg', 'cat.2243.jpg', 'dog.10695.jpg', 'cat.4477.jpg']
['9932.jpg', '4641.jpg', '10238.jpg', '4847.jpg', '9029.jpg', '11939.jpg', '3472.jpg', '397.jpg', '3511.jpg', '1758.jpg']


In [4]:
dictionary = {
    "filename": [],
    "id": [],
    "label": []
}

for i in sorted(img_train):
  dictionary["filename"].append(i)
  dictionary["id"].append(i.split(".")[1])
  dictionary["label"].append(i.split(".")[0])


df = pd.DataFrame(dictionary)
df

,filename,id,label
0,cat.0.jpg,0,cat
1,cat.1.jpg,1,cat
2,cat.10.jpg,10,cat
3,cat.100.jpg,100,cat
4,cat.1000.jpg,1000,cat
...,...,...,...
24995,dog.9995.jpg,9995,dog
24996,dog.9996.jpg,9996,dog
24997,dog.9997.jpg,9997,dog
24998,dog.9998.jpg,9998,dog


In [9]:
X = df[["filename", "label"]]

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")

X_train: (20000, 2), X_test: (5000, 2)


In [10]:
X_train

,filename,label
23311,dog.8479.jpg,dog
23623,dog.876.jpg,dog
1020,cat.10915.jpg,cat
12645,dog.10127.jpg,dog
1533,cat.11377.jpg,cat
...,...,...
21575,dog.6916.jpg,dog
5390,cat.36.jpg,cat
860,cat.10771.jpg,cat
15795,dog.1713.jpg,dog


# Image generator (for VGG16)

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  fill_mode="nearest")


test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_dataframe(dataframe=X_train,
                                                   x_col="filename",
                                                   y_col="label",
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode="binary")

test_generator = test_datagen.flow_from_dataframe(dataframe=X_test,
                                                   x_col="filename",
                                                   y_col="label",
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode="binary")

Found 0 validated image filenames belonging to 0 classes.
Found 0 validated image filenames belonging to 0 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 20000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 5000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


# CNN, що використовує VGG16 в якості згорткової основи

In [ ]:
conv_base = VGG16(weights="imagenet", include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = False

original_dim = (128, 128, 3)
target_size = (150, 150)

In [ ]:
vgg_model = Sequential([
    layers.Input(original_dim),
    layers.Lambda(lambda image: tf.image.resize(image, target_size)),
    conv_base,

    layers.Flatten(),
    layers.Dropout(0.3),

    layers.Dense(512, activation="relu", kernel_initializer="he_uniform"),
    layers.Dense(1, activation="softmax"),
])

In [ ]:
vgg_model.compile(
    loss="binary_crossentropy",
    optimizer=tensorflow.keras.optimizers.Adam(learning_rate=2e-5),
    metrics=["accuracy"])

In [ ]:
vgg_model.summary()

In [ ]:
vgg_history = vgg_model.fit(
    train_generator,
    steps_per_epoch=50,
    epochs=10,
    validation_data=test_generator,
    validation_steps=50
)

# Візуалізація

In [ ]:
# epochs = range(1, len(history.history["accuracy"]) + 1)

# plt.figure(figsize=(8, 4))
# plt.plot(epochs, history.history["accuracy"], ":", label="Training accuracy")
# plt.plot(epochs, history.history["val_accuracy"], label="Validation accuracy")

# plt.title("Training and validation accuracies")
# plt.legend()


# plt.figure(figsize=(8, 4))
# plt.plot(epochs, history.history["loss"], ":", label="Training loss")
# plt.plot(epochs, history.history["val_loss"], label="Validation loss")

# plt.title("Training and validation losses")
# plt.legend()

# plt.show()

# Збереження моделі

In [ ]:
# model.save("/direct/vgg16_basesd_model.hdf5")

# Побудова CNN з Аугментацією

#### Насправді, дані для моделі з VGG16 базою вже аугментовані

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=Adam(learning_rate=1e-4),
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=50,
    epochs=10,
    validation_data=test_generator,
    validation_steps=50
)

# Розпізнавання класу

In [ ]:
n_img = 400
plt.figure(figsize=(2, 2))
plt.imshow(X_test[n_img].reshape(28, 28), cmap=plt.cm.binary)
plt.show()

In [ ]:
x = X_test[n_img]
x = np.expand_dims(x, axis=0)
prediction = model.predict(x)
prediction = np.argmax(prediction[0])
print(f"Номер класу: {prediction}\nНазва: {classes[prediction]}")